In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import detector
import matplotlib.pyplot as plt
import cv2
import skvideo.io as io
import time

In [ ]:
num1 = 254
num2 = 259

im1 = 'imgs/{}_cam-image_array_.jpg'.format(num1)
im2 = 'imgs/{}_cam-image_array_.jpg'.format(num2)
depth1 = 'imgs/{}_cam-depth_array_.png'.format(num1)
depth2 = 'imgs/{}_cam-depth_array_.png'.format(num2)

im1 = cv2.imread(im1, cv2.IMREAD_GRAYSCALE)
im2 = cv2.imread(im2, cv2.IMREAD_GRAYSCALE)
depth1 = cv2.imread(depth1, -1)
depth2 = cv2.imread(depth2, -1)
depth1 = cv2.GaussianBlur(depth1, (9, 7), 0)
depth2 = cv2.GaussianBlur(depth2, (9, 7), 0)

detect = detector.Detector(im1, im2, depth1, depth2, detector='fast')

In [ ]:
kp1, kp2, matches = detect.match()
result_2d = detect.detect_2D(kp1, kp2, matches)
plt.imshow(result_2d, cmap='gray')

In [ ]:
kp1, des1, kp2, des2 = detect.feature_extraction()
matches = detect.fMatcher.knnMatch(des2, des1, k=2)
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
img3 = cv2.drawMatches(detect.img1,kp1,detect.img2,kp2,good[:300],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(12, 6))
plt.imshow(img3)
kp1_3D, kp2_3D = detect.compute3D(good, kp1, kp2)
R, T, mask = detect.estimate_rigid_transform(kp1_3D, kp2_3D)

In [ ]:
# create a point cloud, dim = (h*w, 3)
low, high = 0, np.max(depth1)
pc1 = detect.img_to_3D(low, high)

# project point cloud one onto the projection plane for frame two
pc1_reproj = detect.camMat @ (R @ pc1.T + T)

# convert the point cloud in the local coordinate frame of img2 into a depth array
h, w = detect.depth1.shape[:2]
depth_info = (pc1_reproj[2]).astype(np.uint16)
coords = (pc1_reproj[:2] / pc1_reproj[2]).astype(int)

d_reproj = np.zeros((h, w))
for i in range(pc1_reproj.shape[1]):
    c, r = coords[:, i]
    if 0 <= c < w and 0 <= r < h:
        d_reproj[r, c] = depth_info[i]

# smooth the depth array, apply bias/gain normalization
d1_blur = cv2.GaussianBlur(d_reproj, (9, 7), 0)
d1_dmean = (d1_blur - np.mean(d1_blur)) / np.std(d1_blur)
d2_dmean = (depth2 - np.mean(depth2)) / np.std(depth2)

# compare the reprojected depth array to the original depth two
diff = np.abs(d1_dmean - d2_dmean)
diff = (diff * 255).astype(np.uint8)
result_3d = cv2.threshold(diff,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
plt.imshow(result_3d, cmap='gray')       

In [ ]:
## how to combine the intensity change and depth change to produce better result
## L2 norm, which gives the similar the result as the l1 norm in the following
# result1 = np.sqrt(np.square(result_2d) + np.square(result_3d))
# plt.imshow(result1.astype(float) > 1.2, cmap='gray')

In [ ]:
kp1, kp2, matches = detect.match()
result_3d = detect.detect_3D(kp1, kp2, matches)
plt.imshow(result_3d, cmap='gray')

In [ ]:
res = detect.detect()
plt.imshow(res, cmap='gray')

In [ ]:
# L1 norm
result2 = result_2d * 0.5 + result_3d * 0.5
plt.imshow(result2 > 200, cmap='gray')